In [16]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg
import matplotlib.pyplot as plt
import seaborn as sns

from ASCE import load_combos
from concrete_design.material_properties import elastic_modulus, cracked_moment_inertia
from concrete_design.steel_reinforcement import rebar_df as rebar
from concrete_design.beam_calculations import design_properties as bm
conc_density = 150.0 #pcf

ModuleNotFoundError: No module named 'matplotlib'

<div style="page-break-after: always;"></div>

# Concrete Wall Design (1-sheet reinforcement)
## Inputs

In [4]:
# Materials and Geometry
fc = 2500 #psi
fy = 60000 #psi
b = 12 #in
h = 8 #in
long_reinf = "#5"
dbar = rebar.loc[long_reinf, "Diameter"] #in
nbar = 1 #no. of bars

In [5]:
d = h/2 #in
d

4.0

In [6]:
# Beta1 (stress block multiplier)

if fc <= 4000 and fc >= 2500:
    beta1 = 0.85
elif fc < 8000 and fc > 4000:
    beta1 = 0.85 - 0.05 * (fc - 4000) / 1000
elif fc >= 8000:
    beta1 = 0.65
else:
    print("error!")

beta1

0.85

In [7]:
# Steel Areas

Ast = rebar.loc[long_reinf, "Area"] * nbar #in^2
Ast

0.31

In [8]:
# Stresses and Strains

Es = 29000000 #psi
ec = 0.003 # concrete max strain
ey = fy / Es

def stress_strain(c):
    es = ec * (d - c) / c           # steel strain at tension side
    fs = es * Es                    # steel stress at tension side
    a = beta1 * c                   # compression stress block depth
    return a, es, fs

In [9]:
# Strength reduction factor (ties, not spirals)

def strength_factor():
    if es <= ey:
        phi, classify = 0.65, "compression-controlled"
    elif es >= ey + 0.003:
        phi, classify = 0.90, "tension-controlled"
    elif es > ey and es < ey + 0.003:
        phi, classify = 0.65 + 0.25 * (es - ey) / (0.003), "transition"
    return phi, classify

In [10]:
# Forces and Moments

def forces_moments():
    T = Ast * min(fy, fs)               # tension force steel at tension side (lbs)
    Cc = 0.85 * fc * a * b              # compression force in concrete (lbs)

    Pn = Cc - T                         # Nominal axial load capacity (lbs)
    phiPn = phi * Pn                    # Ultimate axial load capacity (kips)

    # Solve for eccentricity by taking moment about T:
    # Sum of moments at T: Pn(ecc) - Cc(d - a/2) = 0
    ecc = Cc*(d - a/2) / Pn             #in (eccentricity)
    Mn = Pn * ecc                       # Nominal moment capacity (in-lbs)
    phiMn = phi * Mn                    # Ultimate moment capacity (in-lbs)

    return ecc, Pn, phiPn, Mn, phiMn

In [11]:
# Solve axial load and moment at varying 'c' values

nom_load = []
ult_load = []
nom_moment = []
ult_moment = []
c_value = []
a_value = []
eccentricity = []
phi_factor = []
failure_mode = []
steel_strain = []

c_values = np.arange(1.5*h, 0, -0.01)
for c in c_values:
    a, es, fs = stress_strain(c)
    phi, classify = strength_factor()
    ecc, Pn, phiPn, Mn, phiMn = forces_moments()
    if ecc <= 1.5*h:
        nom_load.append(round(Pn))
        ult_load.append(round(phiPn))
        nom_moment.append(round(Mn))
        ult_moment.append(round(phiMn))
        c_value.append(round(c, 3))
        a_value.append(round(a, 4))
        eccentricity.append(round(ecc, 4))
        phi_factor.append(round(phi, 2))
        failure_mode.append(classify)
        steel_strain.append(round(es, 4))

# change units
nom_load = [i / 1000 for i in nom_load]             # kips
ult_load = [i / 1000 for i in ult_load]             # kips
nom_moment = [i / 12 for i in nom_moment]           # lb-ft
ult_moment = [i / 12 for i in ult_moment]           # lb-ft
        
# Save the above calculated data in a dataframe
dict = {
    "c value": c_value,
    "a values (in)": a_value,
    "ecc (in)": eccentricity,
    "es (in/in)": steel_strain,
    "Pn (kips)": nom_load,
    "phiPn (kips)": ult_load,
    "Mn (ft-lbs)": nom_moment,
    "phiMn (ft-lbs)": ult_moment,
    "phi": phi_factor,
    "failure mode": failure_mode
}

df = pd.DataFrame(dict)

# Print the last 10 rows of the dataframe to verify
df.head(10)
#df.tail(10)
#df[(df["phiMn (ft-lbs)"] > -10) & (df["phiMn (ft-lbs)"] < 10)]
#df[(df["ecc (in)"] > -0.1) & (df["ecc (in)"] < 0.1)]

,c value,a values (in),ecc (in),es (in/in),Pn (kips),phiPn (kips),Mn (ft-lbs),phiMn (ft-lbs),phi,failure mode
0,12.00,10.2000,-1.0289,-0.002,278.080,180.752,-23842.500000,-15497.583333,0.65,compression-controlled
1,11.99,10.1915,-1.0249,-0.002,277.856,180.606,-23730.583333,-15424.916667,0.65,compression-controlled
2,11.98,10.1830,-1.0209,-0.002,277.631,180.460,-23618.833333,-15352.250000,0.65,compression-controlled
3,11.97,10.1745,-1.0169,-0.002,277.407,180.315,-23507.250000,-15279.666667,0.65,compression-controlled
4,11.96,10.1660,-1.0129,-0.002,277.183,180.169,-23395.750000,-15207.250000,0.65,compression-controlled
5,11.95,10.1575,-1.0089,-0.002,276.959,180.023,-23284.500000,-15134.916667,0.65,compression-controlled
6,11.94,10.1490,-1.0049,-0.002,276.734,179.877,-23173.333333,-15062.666667,0.65,compression-controlled
7,11.93,10.1405,-1.0009,-0.002,276.510,179.732,-23062.333333,-14990.500000,0.65,compression-controlled
8,11.92,10.1320,-0.9969,-0.002,276.286,179.586,-22951.500000,-14918.500000,0.65,compression-controlled
9,11.91,10.1235,-0.9929,-0.002,276.061,179.440,-22840.833333,-14846.500000,0.65,compression-controlled


In [12]:
# Calculate the maximum allowable axial load (concentric)

phi = 0.65
Ag = b*h #in^2
As = Ast #in^2
Pmax = phi * 0.80 * (0.85*fc*(Ag-As) + As*fy) / 1000
f"Max allowable concentric axial load = {round(Pmax, 2)} kips"

'Max allowable concentric axial load = 115.41 kips'

In [13]:
# Calculate the maximum allowable flexural demand

phi = 0.9
d = h/2
a = As * fy / (0.85 * fc * b)
Mn = As * fy * (d - a/2)
Mu = phi * Mn / 12 #ft-lbs

print(f"Mmax = {round(Mu, 2)} lb-ft")

Mmax = 5071.24 lb-ft


In [14]:
# Plotting the P-M Interaction Diagram

sns.set_style("darkgrid")
fig, ax = plt.subplots(figsize=(13,7))
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlesize=15)
ax = sns.scatterplot(x="Mn (ft-lbs)", y="Pn (kips)", data=df, color="g", label="Nominal")
sns.scatterplot(x="phiMn (ft-lbs)", y="phiPn (kips)", data=df, color="brown", label="Ultimate")
ax.set_xlabel("Moment (M, ft-lbs)")
ax.set_ylabel("Axial (P, kips)")
plt.title("P-M Interaction Diagram")

plt.ylim(0, 1.1*max(dict["Pn (kips)"]))
plt.xlim(0, 1.1*max(dict["Mn (ft-lbs)"]))
plt.axhline(Pmax, color="red")
plt.text(100, Pmax+2, "Pmax")

plt.axvline(Mu, color="black")
plt.text(Mu+2, 20, "Mmax")

plt.show()

NameError: name 'sns' is not defined